In [4]:
import pandas as pd
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))  # adds parent directory

from common.audit import AuditLogReader

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

log_rdr = AuditLogReader("/home/mike/workspace/soa-ilec/soa-ilec/mcp_agent_work")
branch_res = log_rdr._traverse_branch("bdea8c99-96be-4d17-a7ba-7649b614dcd4")
branch_res

{'type': 1,
 'next': {'type': 2,
  'next': {'type': 2,
   'next': {'type': 2,
    'next': {'type': 2,
     'next': None,
     'entry': {'last_workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
      'workspace_id': 'bdea8c99-96be-4d17-a7ba-7649b614dcd4',
      'tool_name': 'cmd_run_inference',
      'args': ['ul_train_data', 'ul_train_data_model_preds'],
      'result': {'success': True,
       'result': 'created dataset ul_train_data_model_preds with predictions contained in MODEL_PRED column.'}},
     'workspace_id': 'bdea8c99-96be-4d17-a7ba-7649b614dcd4'},
    'entry': {'last_workspace_id': '312ac4f8-0160-47ca-a8c5-1bb7592d5434',
     'workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
     'tool_name': 'cmd_glmnet',
     'args': ['ul_train_data',
      ['Gender', 'Attained_Age', 'Smoker_Status', 'Face_Amount_Band'],
      ['Gender * splines::ns(Attained_Age, knots=c(30, 40, 50, 60, 70, 80), Boundary.knots=c(17, 95))',
       'Smoker_Status',
       'Face_Amount_Band'],
    

In [8]:
log_rdr._traverse_tree(branch_res["workspace_id"])

{'workspace_id': '91e633c1-9a04-4f23-803d-67af9655a083',
 'type': 1,
 'next': [{'workspace_id': '488d8c35-8f1a-4e08-b28c-97f68e1e7bed',
   'type': 2,
   'next': [{'workspace_id': '312ac4f8-0160-47ca-a8c5-1bb7592d5434',
     'type': 2,
     'next': [{'workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
       'type': 2,
       'next': [{'workspace_id': 'bdea8c99-96be-4d17-a7ba-7649b614dcd4',
         'type': 2,
         'next': [],
         'entry': {'last_workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
          'workspace_id': 'bdea8c99-96be-4d17-a7ba-7649b614dcd4',
          'tool_name': 'cmd_run_inference',
          'args': ['ul_train_data', 'ul_train_data_model_preds'],
          'result': {'success': True,
           'result': 'created dataset ul_train_data_model_preds with predictions contained in MODEL_PRED column.'}}}],
       'entry': {'last_workspace_id': '312ac4f8-0160-47ca-a8c5-1bb7592d5434',
        'workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
       